In [2]:
%pip install azureml-mlflow==1.52.0

Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install --upgrade torch
%pip install transformers[torch]
%pip install --upgrade mlflow
%pip install --upgrade transformers=49.2.0
%pip install --upgrade datasets==2.9.0

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
ERROR: Invalid requirement: 'transformers=49.2.0'
Hint: = is not a valid operator. Did you mean == ?
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install evaluate

Note: you may need to restart the kernel to use updated packages.


In [5]:
%pip install mlflow azureml-mlflow

Note: you may need to restart the kernel to use updated packages.


In [6]:
import os 
import mlflow
import pandas as pd
import numpy as np
from datasets import load_dataset
import evaluate
from azureml.core import Workspace
from transformers import AutoModelForSequenceClassification,AutoTokenizer,TrainingArguments,Trainer
import pickle
import time
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential,AzureCliCredential

subscription_id = '80c77c76-74ba-4c8c-8229-4c3b2957990c'
resource_group = 'sonata-test-rg'
workspace_name = 'sonata-test-ws'

credential = AzureCliCredential()
ws = Workspace(subscription_id, resource_group, workspace_name)

workspace_ml_client = MLClient(
        credential, subscription_id, resource_group, ws
    )

# store the mlflow results in azure ml workspace
mlflow.set_tracking_uri(ws.get_mlflow_tracking_uri())


/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
# genrating a unique timestamp that can be used for names and versions that need to be unique
timestamp = str(int(time.time()))

In [8]:
import mlflow

In [9]:
EXPERIMENT_NAME = "Experiment with RoBerta_HF"
mlflow.set_experiment(EXPERIMENT_NAME)

<Experiment: artifact_location='', creation_time=1689074342754, experiment_id='2f93ed0c-13c6-4c16-bc99-4c8ad255f092', last_update_time=None, lifecycle_stage='active', name='Experiment with RoBerta_HF', tags={}>

In [10]:
Model_Name = 'SamLowe/roberta-base-go_emotions'

In [11]:
# with mlflow.start_run() as run:
    # loading the train & test data with 1000 rows
test_dataset = load_dataset("glue","cola", split="test").shuffle().select(range(100))
train_dataset = load_dataset("glue","cola", split="train").shuffle().select(range(100))

Found cached dataset glue (/home/azureuser/.cache/huggingface/datasets/glue/cola/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
Found cached dataset glue (/home/azureuser/.cache/huggingface/datasets/glue/cola/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


In [12]:
#checking the features
features = train_dataset.features
print(features)
    

{'sentence': Value(dtype='string', id=None), 'label': ClassLabel(names=['unacceptable', 'acceptable'], id=None), 'idx': Value(dtype='int32', id=None)}


In [13]:
#Understanding the lables in datasets
print(features["label"].int2str(0)) 

unacceptable


In [14]:
#converting id to labels
id2label = {idx:features["label"].int2str(idx) for idx in range(2)}
print(id2label)

{0: 'unacceptable', 1: 'acceptable'}


In [15]:
#converting labels to ids
label2id = {value:key for key,value in id2label.items()}
print(label2id)

{'unacceptable': 0, 'acceptable': 1}


In [16]:
#loding the basemodel
model = AutoModelForSequenceClassification.from_pretrained(
        Model_Name
        # ,num_labels = 2,
        # id2label = id2label,
        # label2id = label2id
        )

In [17]:
#Preprocessing the data to suit with model compatability
tokenizer = AutoTokenizer.from_pretrained(Model_Name)

In [18]:
%pip install xformers

Note: you may need to restart the kernel to use updated packages.


In [19]:
#Save the model in mlflow with tokenizer
base_model = mlflow.transformers.log_model(
        transformers_model={"model": model, "tokenizer": tokenizer},
        task="text-classification",
        artifact_path="Models/basemodel",
        input_example=["base model loaded directly from transformer's"],
        registered_model_name="RoBerta_HF"
        )

/tmp/ipykernel_5986/545235599.py:2: FutureWarning: The 'transformers' MLflow Models integration is known to be compatible with the following package version ranges: ``4.25.1`` -  ``4.29.2``. MLflow Models integrations with transformers may not succeed when used with package versions outside of this range.
  base_model = mlflow.transformers.log_model(
/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/mlflow/models/model.py:562: FutureWarning: The 'transformers' MLflow Models integration is known to be compatible with the following package version ranges: ``4.25.1`` -  ``4.29.2``. MLflow Models integrations with transformers may not succeed when used with package versions outside of this range.
  flavor.save_model(path=local_path, mlflow_model=mlflow_model, **kwargs)
/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered m

In [20]:
# model_uri = f"models:/Roberta_HF/4"
# model = mlflow.pyfunc.load_model(model_uri)

In [21]:
# # need to specify model versions until the bug to support fetching the latest version using latest label is fixed
# model_details = [
#     {"name": "Roberta_HF"},
#     # please prepare appropriate dataset and config in similar way to run evaluation on this dataset
#     #     {"name": "finiteautomata-bertweet-base-sentiment-analysis"},
# ]

In [22]:
# models = []
# for model in client.search_registered_models():
#     print(f"{model.name}")
#     print(list(model.latest_versions))
# # for model in model_details:
# #     client.get_registered_model(model.name)
# #     reg_model = list(registry_ml_client.models.list(name=model["name"]))[0]
# #     print(reg_model.id)
# #     models.append({**model, "version": reg_model.version})

In [23]:
# base_model.model_uri

In [24]:
# client = mlflow.tracking.MlflowClient()
# client

In [25]:
# for model in client.search_registered_models():
#     print(f"{model.name}")

In [26]:
# train_model=client.get_model_version("Roberta_HF", version=1)
# train_model.id

In [27]:
def tokenizer_function(examples):
        return tokenizer(examples['sentence'],padding='max_length',truncation=True)


In [28]:
train_dataset = train_dataset.map(tokenizer_function,batched=True)
test_dataset = test_dataset.map(tokenizer_function,batched = True)
    
print(train_dataset)

100%|██████████| 1/1 [00:00<00:00, 39.95ba/s]


Dataset({
    features: ['sentence', 'label', 'idx', 'input_ids', 'attention_mask'],
    num_rows: 100
})


In [29]:
#converting the data into pandas df
train_dataset_df = train_dataset.to_pandas() 
print(train_dataset_df)

                                             sentence  label   idx   
0    Whenever Bill smokes, Susan hates him much more.      1   223  \
1                 I saw more of the man than you did.      1  5397   
2   That Bill's breath smells of onions bothers Erin.      1  5744   
3                              We made them are rude.      0  4117   
4                    That was why she looked so nice.      1  5099   
..                                                ...    ...   ...   
95                 The bed was fallen on by the dust.      0  6135   
96  David constructed the mansion from bricks into...      0  2949   
97               Susan gave the ball halfway to Bill.      0  2032   
98                                     The cup broke.      1  2180   
99            Carla poured lemonade into the pitcher.      1  2135   

                                            input_ids   
0   [0, 32395, 1585, 40383, 6, 6470, 27701, 123, 2...  \
1   [0, 100, 794, 55, 9, 5, 313, 87, 47, 222,

In [30]:
# train_dataset_df["label"] = train_dataset_df["label"].astype(int)

In [31]:
# type(test_dataset["label"])

In [32]:
#save the train & test dataset
test_dataset.to_csv("glue_test_data.csv")
train_dataset.to_csv("glue_train_data.csv")
    
mlflow.log_artifact("glue_test_data.csv", artifact_path = "Datasets")
mlflow.log_artifact("glue_train_data.csv", artifact_path = "Datasets")
    

    

Creating CSV from Arrow format: 100%|██████████| 1/1 [00:00<00:00,  6.44ba/s]


In [33]:
import numpy as np
from datasets import load_dataset
from transformers import (
    BertForSequenceClassification,
    BertTokenizerFast,
    TrainingArguments,
    Trainer,
    EvalPrediction,
)

metric = evaluate.load("accuracy")


def compute_metrics(eval_preds: EvalPrediction):
    x, y = eval_preds
    preds = np.argmax(x, -1)
    return metric.compute(predictions=preds, references=y)



In [34]:
# model_uri = f"models:/Roberta_HF/1"
# loaded_model = mlflow.pyfunc.load_model(model_uri)

In [35]:
train_dataset.set_format(type="torch", columns=['sentence', 'label', 'idx', 'input_ids', 'attention_mask'])
train_dataset.format['type']

'torch'

In [36]:
Training_args = TrainingArguments(
    hub_model_id='RoBerta_HF:5',
    num_train_epochs = 1,
    output_dir= ".",
    logging_steps=500,
    save_strategy="epoch",
    push_to_hub=False,
    
)

In [37]:
trainer = Trainer(
    model = model,
    args = Training_args,
    train_dataset = train_dataset,
    eval_dataset = test_dataset,
    compute_metrics=compute_metrics
    )
    

#train the model and print the metrics related to trainer & accuracy 
# result = trainer.train()

In [38]:
result = trainer.train()

/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


ValueError: Target size (torch.Size([8])) must be the same as input size (torch.Size([8, 28]))

In [39]:
mlflow.end_run()

In [1]:
# create a deployment
demo_deployment = ManagedOnlineDeployment(
    name="RoBertaEndpoint",
    endpoint_name=online_endpoint_name,
    model='RoBerta_HF:3',
    instance_type="Standard_DS2_v2",
    instance_count=1,
    request_settings=OnlineRequestSettings(
        request_timeout_ms=60000,
    ),
)
workspace_ml_client.online_deployments.begin_create_or_update(demo_deployment).wait()
endpoint.traffic = {"RoBertaEndpoint": 100}
workspace_ml_client.begin_create_or_update(endpoint).result()

NameError: name 'ManagedOnlineDeployment' is not defined

In [ ]:
import json
import os

# pick 1 random row
sample_df = train_df.sample(1)
# create a json object with the key as "inputs" and value as a list of values from the en column of the sample_df dataframe
sample_json = {"inputs": sample_df["text"].tolist()}
# save the json object to a file named sample_score.json in the ./polyglot_ner-dataset folder
test_json = {"inputs": {"input_string": sample_df["text"].tolist()}}
# save the json object to a file named sample_score.json in the ./polyglot_ner-dataset folder
with open(os.path.join(".", "ML Flows", "sample_score.json"), "w") as f:
    json.dump(test_json, f)
sample_df.head()

In [ ]:
# score the sample_score.json file using the online endpoint with the azureml endpoint invoke method
response = workspace_ml_client.online_endpoints.invoke(
    endpoint_name=online_endpoint_name,
    deployment_name="RoBertaEndpoint",
    request_file="./ML Flows/sample_score.json",
)
print("raw response: \n", response, "\n")
# convert the json response to a pandas dataframe
response_df = pd.read_json(response)
response_df.head()

In [ ]:
workspace_ml_client.online_endpoints.begin_delete(name=online_endpoint_name).wait()

In [ ]:
# mlflow.pyfunc.save_model(path=base_model.model_uri, python_model="Roberta_HF")